# Train VGGNet and FCN as Segmentation model
## Breast-Ultrasound-Segmentation

## About Dataset
Breast cancer is one of the most common causes of death among women worldwide. Early detection helps in reducing the number of early deaths. The data reviews the medical images of breast cancer using ultrasound scan. Breast Ultrasound Dataset is categorized into three classes: normal, benign, and malignant images. Breast ultrasound images can produce great results in classification, detection, and segmentation of breast cancer when combined with machine learning.

### Data
The data collected at baseline include breast ultrasound images among women in ages between 25 and 75 years old. This data was collected in 2018. The number of patients is 600 female patients. The dataset consists of 780 images with an average image size of 500*500 pixels. The images are in PNG format. The ground truth images are presented with original images. The images are categorized into three classes, which are normal, benign, and malignant.

If you use this dataset, please cite:
Al-Dhabyani W, Gomaa M, Khaled H, Fahmy A. Dataset of breast ultrasound images. Data in Brief. 2020 Feb;28:104863. DOI: 10.1016/j.dib.2019.104863.

## Imports

In [1]:
import os

import pyrootutils

root = pyrootutils.setup_root(
    search_from=os.path.dirname(os.getcwd()),
    indicator=[".git", "pyproject.toml"],
    pythonpath=True,
    dotenv=True,
)

if os.getenv("DATA_ROOT") is None:
    os.environ["DATA_ROOT"] = f"{root}/data"

In [2]:
import torch
import torch.nn as nn

# Setup device-agnostic code
if torch.cuda.is_available():
    DEVICE = "cuda"  # NVIDIA GPU
    print("GPU Found!!")
else:
    raise Exception("No GPU Found!!")

GPU Found!!


In [3]:
import hydra
from hydra import compose, initialize

In [4]:
# # auto reload dotenv
%load_ext dotenv
%dotenv

# auto reload libs
%load_ext autoreload
%autoreload 2

## Paths setup

In [5]:
from omegaconf import DictConfig, OmegaConf

# Register a resolver for torch dtypes
OmegaConf.register_new_resolver("torch_dtype", lambda name: getattr(torch, name))

with initialize(config_path="../configs", job_name="training_setup", version_base=None):
    cfg: DictConfig = compose(config_name="train.yaml")
    # print(OmegaConf.to_yaml(cfg))
    print(cfg.models)

{'optimizer': {'_target_': 'torch.optim.Adam', '_partial_': True, 'lr': 0.001, 'weight_decay': 0.0, 'betas': [0.9, 0.999]}, 'scheduler': {'func': None}, 'model': {'_target_': 'src.models.vggnet_fcn_segmentation_model.VGGNetFCN16SegmentationModel', 'num_classes': 3}}


In [6]:
os.chdir(root)

## Loading Dataset

In [7]:
data_module = hydra.utils.instantiate(cfg.datamodule)

class_weights = data_module.class_weights
class_names = data_module.classes
num_classes = len(class_names)
class_names, num_classes, class_weights

(['normal', 'malignant', 'benign'], 3, tensor([1.9774, 1.2494, 0.5903]))

In [8]:
import torch.optim as optim
from torch.utils.data import DataLoader, Subset

# # 1. Select a very small subset of your data
train_dataset, val_dataset = data_module.split_and_preprocess_datasets()

subset_indices = list(range(10))  # Select first 10 samples
train_small_dataset = Subset(train_dataset, subset_indices)
val_small_dataset = Subset(val_dataset, subset_indices)
train_dl = DataLoader(train_small_dataset, batch_size=5, shuffle=False)
val_dl = DataLoader(val_small_dataset, batch_size=5, shuffle=False)

In [9]:
for images, targets in train_dl:
    print(images.shape, targets["masks"].shape, targets["labels"].shape)

    print(f"images:{images.dtype}, {images[0].min()}, {images[0].max()}")
    print(
        f'masks {targets["masks"].dtype}, {targets["masks"][0].min()}, {targets["masks"][0].max()}'
    )
    print(
        f'labels {targets["labels"].dtype}, {targets["labels"].min()}, {targets["labels"].max()}'
    )
    break

torch.Size([5, 3, 224, 224]) torch.Size([5, 1, 224, 224]) torch.Size([5])
images:torch.float32, -2.1179039478302, 2.6225709915161133
masks torch.uint8, 0, 1
labels torch.int64, 1, 2


In [10]:
for _images, _targets in val_dl:
    print(_images.shape, _targets["masks"].shape, _targets["labels"].shape)

    print(f"images:{_images[0].dtype}, {_images[0].min()}, {_images[0].max()}")
    print(f'masks {_targets["masks"].dtype}, {_targets["masks"].min()}, {_targets["masks"].max()}')
    print(
        f'labels {_targets["labels"].dtype}, {_targets["labels"].min()}, {_targets["labels"].max()}'
    )
    break

torch.Size([5, 3, 224, 224]) torch.Size([5, 1, 224, 224]) torch.Size([5])
images:torch.float32, -2.1179039478302, 2.5354254245758057
masks torch.uint8, 0, 1
labels torch.int64, 0, 2


## Loading and training the FCN8 model 

In [11]:
from src.utils.gpu_utils import DeviceDataLoader, get_default_device, to_device

torch.cuda.empty_cache()
device = get_default_device()

gpu_weights = to_device(class_weights, device)

In [12]:
from src.models.vggnet_fcn16_segmentation_model import VGGNetFCN16SegmentationModel

model = VGGNetFCN16SegmentationModel(
    num_classes=num_classes, vggnet_type="vgg16", class_weights=gpu_weights
)
model = torch.compile(model)
model

OptimizedModule(
  (_orig_mod): VGGNetFCN16SegmentationModel(
    (encoder): VGGNetEncoder(
      (vgg): VGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): ReLU(inplace=True)
          (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (6): ReLU(inplace=True)
          (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): ReLU(inplace=True)
          (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (11): ReLU(inplace=True)
          (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (13

In [13]:
for images, labels in train_dl:
    # print(images.shape, labels)
    out = model(images)
    # l = labels['labels'][0]
    # print(l , torch.argmax(l))
    print(out["labels"][0], out["labels"].shape)
    print(out["masks"][0])
    print(out["masks"].shape)
    break

tensor([-0.0178,  0.0420,  0.0440], grad_fn=<SelectBackward0>) torch.Size([5, 3])
tensor([[[-0.6220,  0.7259, -1.3855,  ...,  0.5466, -0.3727, -0.1547],
         [ 1.5213,  1.3550,  1.0672,  ..., -0.1229,  0.0301, -0.4407],
         [-0.7953,  1.3074,  0.1862,  ...,  0.8725, -1.1757, -0.1379],
         ...,
         [ 0.5386, -1.7544,  0.3659,  ...,  1.2633,  0.2619, -0.2495],
         [-0.2249, -0.2340,  0.3166,  ...,  0.4324, -0.2928, -0.3038],
         [-0.0977,  0.1719,  0.7858,  ...,  0.1595, -0.0399, -0.2169]],

        [[-0.6621,  0.3084, -1.6984,  ..., -0.3331, -0.2171,  0.1832],
         [ 2.4377,  4.0421,  3.9546,  ...,  1.5136,  0.5262,  0.0863],
         [ 0.6439,  0.9135,  1.3605,  ...,  1.4076,  1.1149,  0.2864],
         ...,
         [ 0.1575, -0.9485,  1.1830,  ..., -1.1196, -0.1106, -0.5340],
         [ 0.0803, -0.5012, -0.6488,  ...,  0.6220,  0.0528, -0.2015],
         [-0.2362,  0.4922,  0.4942,  ..., -0.1537, -0.2174, -0.6524]],

        [[ 0.2847, -0.2758, -0.120

### Evaluate function

In [14]:
@torch.no_grad()
def evaluate(model, val_dl) -> dict[str, float]:
    model.eval()  # set model to evaluate mode
    outputs = [model.validation_step(batch) for batch in val_dl]
    return model.validation_epoch_end(outputs)

In [15]:
def fit(
    model,
    train_dataloader,
    validation_dataloader,
    epochs: int = 2,
    lr: float = 1e-3,
    opt_func=torch.optim.Adam,
) -> list[dict[str, float]]:
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        train_accuracies = []

        train_losses = []
        for batch in train_dataloader:
            step_output = model.training_step(batch)
            loss = step_output["loss"]
            train_acc = step_output["train_acc"]
            # Detach loss and accuracy before appending to avoid holding onto computation graph
            train_losses.append(loss.detach())
            train_accuracies.append(train_acc.detach())  # Assuming train_acc is a tensor

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation Phase
        result = evaluate(model, validation_dataloader)
        result["train_loss"] = torch.stack(train_losses).mean().item()
        result["train_acc"] = torch.stack(train_accuracies).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [16]:
# evaluate(model, val_dl)

## GPU Training Setup

### Utils preparation

## Moving data and model into memory

### Train only Classification head 

In [17]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device)
# train_dl.device

OptimizedModule(
  (_orig_mod): VGGNetFCN16SegmentationModel(
    (encoder): VGGNetEncoder(
      (vgg): VGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): ReLU(inplace=True)
          (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (6): ReLU(inplace=True)
          (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): ReLU(inplace=True)
          (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (11): ReLU(inplace=True)
          (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (13

In [ ]:
num_epochs = 20  # Train for significantly more epochs than usual

# history = fit(
#     model=model,
#     train_dataloader=train_dl,
#     validation_dataloader=val_dl,
#     epochs=num_epochs,
#     lr=1e-4,
#     opt_func=optim.Adam,
# )

In [19]:
from src.utils.visualizations import plot_losses

# plot_losses(history)

## Image Segmentation

In [20]:
import enum


class MetricKey(enum.Enum):
    # classification
    CLS_LOSS = "cls_loss"
    CLS_TRAIN_ACC = "cls_train_acc"
    VAL_CLS_LOSS = "cls_val_loss"
    VAL_CLS_ACC = "cls_val_acc"

    # segmentation
    MASKS_LOSS = "masks_loss"
    MASKS_DICE_SCORE = "masks_dice_score"
    VAL_MASK_LOSS = "val_mask_loss"
    VAL_DICE_SCORE = "val_dice_score"


def fit_seg(
    model,
    train_dataloader,
    validation_dataloader,
    epochs: int = 2,
    lr: float = 1e-3,
    opt_func=torch.optim.Adam,
) -> list[dict[str, float]]:
    history = []
    result = {}
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_losses = []
        masks_dice_sc = []
        train_accuracies = []
        loss = []
        masks_losses = []
        for batch in train_dataloader:
            step_output = model.training_step(batch)
            masks_loss = step_output[f"{MetricKey.MASKS_LOSS.value}"]
            _masks_dice_sc = step_output[f"{MetricKey.MASKS_DICE_SCORE.value}"]

            train_acc = step_output[f"{MetricKey.CLS_TRAIN_ACC.value}"]
            loss = step_output[f"{MetricKey.CLS_LOSS.value}"]
            # Detach loss and accuracy before appending to avoid holding onto computation graph
            train_losses.append(loss.detach())
            masks_losses.append(masks_loss.detach())  # Assuming train_acc is a tensor
            masks_dice_sc.append(_masks_dice_sc)  # Assuming train_acc is a tensor
            train_accuracies.append(train_acc.detach())  # Assuming train_acc is a tensor

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Validation Phase
        result = evaluate(model, validation_dataloader)
        result[f"{MetricKey.CLS_LOSS.value}"] = torch.stack(train_losses).mean().item()
        result[f"{MetricKey.MASKS_LOSS.value}"] = torch.stack(masks_losses).mean().item()
        result[f"{MetricKey.MASKS_DICE_SCORE.value}"] = torch.stack(masks_dice_sc).mean().item()
        result[f"{MetricKey.CLS_TRAIN_ACC.value}"] = torch.stack(train_accuracies).mean().item()
        model.epoch_end(epoch, result)
        # history.append(result)
    return history

In [24]:
# TODO: try using dice loss
history = fit_seg(
    model=model,
    train_dataloader=train_dl,
    validation_dataloader=val_dl,
    epochs=num_epochs + 50,
    lr=1e-5,
    opt_func=optim.Adam,
)

Epoch [0], masks_loss: 1.1372,masks_dice_score: nan,
Epoch [1], masks_loss: 1.1374,masks_dice_score: nan,
Epoch [2], masks_loss: 1.1368,masks_dice_score: nan,
Epoch [3], masks_loss: 1.1382,masks_dice_score: nan,
Epoch [4], masks_loss: 1.1373,masks_dice_score: nan,
Epoch [5], masks_loss: 1.1363,masks_dice_score: nan,
Epoch [6], masks_loss: 1.1371,masks_dice_score: nan,
Epoch [7], masks_loss: 1.1372,masks_dice_score: nan,
Epoch [8], masks_loss: 1.1390,masks_dice_score: nan,
Epoch [9], masks_loss: 1.1384,masks_dice_score: nan,
Epoch [10], masks_loss: 1.1380,masks_dice_score: nan,
Epoch [11], masks_loss: 1.1390,masks_dice_score: nan,
Epoch [12], masks_loss: 1.1392,masks_dice_score: nan,
Epoch [13], masks_loss: 1.1393,masks_dice_score: nan,
Epoch [14], masks_loss: 1.1386,masks_dice_score: nan,
Epoch [15], masks_loss: 1.1397,masks_dice_score: nan,
Epoch [16], masks_loss: 1.1391,masks_dice_score: nan,
Epoch [17], masks_loss: 1.1411,masks_dice_score: nan,
Epoch [18], masks_loss: 1.1411,masks_d

In [ ]:
import matplotlib.pyplot as plt

train_losses = [x["train_loss"] for x in history]

plt.plot(train_losses, "-bx")

plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid()
plt.legend(["train_loss"])
plt.title("Loss vs. NO. of epochs")

In [ ]:
import torch
import torch.nn.functional as F

# Example predictions (logits) and targets
predictions = torch.randn(32, 3, 224, 224)
targets = torch.randint(0, 3, (32, 224, 224), dtype=torch.long)

targets.shape, predictions.shape

In [ ]:
# Calculate the loss using the functional form
loss = F.cross_entropy(predictions, targets)
print(loss.item())

# You can also specify arguments directly in the function call
loss_with_reduction = F.cross_entropy(predictions, targets, reduction="sum")
print(loss_with_reduction.item())